## Books Database Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
!mongoimport --type csv -d books_db -c books --headerline --drop Resources/Books.csv
!mongoimport --type csv -d books_db -c users --headerline --drop Resources/Users.csv
!mongoimport --type csv -d books_db -c ratings --headerline --drop Resources/Ratings.csv
!mongoimport --type csv -d books_db -c geodata --headerline --drop Resources/GeoLoc.csv

2023-12-07T23:19:25.959-0500	connected to: mongodb://localhost/
2023-12-07T23:19:25.960-0500	dropping: books_db.books
2023-12-07T23:19:26.050-0500	2035 document(s) imported successfully. 0 document(s) failed to import.
2023-12-07T23:19:26.193-0500	connected to: mongodb://localhost/
2023-12-07T23:19:26.193-0500	dropping: books_db.users
2023-12-07T23:19:26.279-0500	3583 document(s) imported successfully. 0 document(s) failed to import.
2023-12-07T23:19:26.434-0500	connected to: mongodb://localhost/
2023-12-07T23:19:26.434-0500	dropping: books_db.ratings
2023-12-07T23:19:26.548-0500	4786 document(s) imported successfully. 0 document(s) failed to import.
2023-12-07T23:19:26.693-0500	connected to: mongodb://localhost/
2023-12-07T23:19:26.694-0500	dropping: books_db.geodata
2023-12-07T23:19:26.729-0500	48 document(s) imported successfully. 0 document(s) failed to import.


In [4]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'books_db', 'config', 'local', 'met', 'uk_food']

In [5]:
# assign the uk_food database to a variable name
db = mongo['books_db']

In [6]:
db['merged'].drop()
db['merged_ratings_users'].drop()
db['merged_users_geo'].drop()

In [7]:
# review the collections in our new database
db.list_collection_names()

['books', 'ratings', 'geodata', 'users']

In [8]:
# assign the collection to a variable
books=db['books']
ratings=db['ratings']
users=db['users']
geodata=db['geodata']

In [9]:
print("Example of a record within 'books' collection: ")
pprint(books.find_one({}))
print("\n Example of a record within 'ratings' collection:")
pprint(ratings.find_one({}))
print("\n Example of a record within 'users' collection:")
pprint(users.find_one({}))
print("\n Example of a record within 'geodata' collection:")
pprint(geodata.find_one({}))

Example of a record within 'books' collection: 
{'': 0,
 'Book-Author': 'Mark P. O. Morford',
 'Book-Title': 'Classical Mythology',
 'ISBN': 195153448,
 'Publisher': 'Oxford University Press',
 'Year-Of-Publication': 2002,
 '_id': ObjectId('6572994d941cd578e6ba9b92')}

 Example of a record within 'ratings' collection:
{'': 353,
 'Book-Rating': 8,
 'ISBN': 3596214629,
 'User-ID': 276994,
 '_id': ObjectId('6572994e38e0654519517cf2')}

 Example of a record within 'users' collection:
{'': 11,
 'Location': 'united states',
 'User-ID': 12,
 '_id': ObjectId('6572994e599371f4406433a5')}

 Example of a record within 'geodata' collection:
{'': 9,
 'Lat': 50.6288684,
 'Long': 3.785318028,
 '_id': ObjectId('6572994ee75361178346e3a2'),
 'country': 'austria'}


In [10]:
# merge all the data
pipeline = [{
    '$lookup': {
      'from': 'geodata',
      'localField': 'Location',
      'foreignField': 'country',
      'as': 'geo_data'
    }},
    {
    '$project': {
      '_id': 0,
      'Age': 1,
      'Location': 1,
      'User-ID': 1,
      'Geo-Data': { '$arrayElemAt': ['$geo_data', 0]}
    }}]

merged_users_geo_cursor=users.aggregate(pipeline)

for i in merged_users_geo_cursor:
    db['merged_users_geo'].insert_one(i)

pipeline = [{
    '$lookup': {
      'from': 'merged_users_geo',
      'localField': 'User-ID',
      'foreignField': 'User-ID',
      'as': 'user_data'
    }},
    {
    '$project': {
      '_id': 0,
      'Book-Rating': 1,
      'ISBN': 1,
      'User-ID': 1,
      'User':  { '$arrayElemAt': ['$user_data', 0] }
    }}]

merged_ratings_users_cursor=ratings.aggregate(pipeline)

for i in merged_ratings_users_cursor:
    db['merged_ratings_users'].insert_one(i)
    
pipeline = [{
    '$lookup': {
      'from': 'merged_ratings_users',
      'localField': 'ISBN',
      'foreignField': 'ISBN',
      'as': 'book_ratings_users'
    }
  },
  {
    '$project': {
      '_id': 0,
      'Book-Author': 1,
      'Book-Title': 1,
      'ISBN': 1,
      'Publisher': 1,
      'Year-Of-Publication': 1,
      'Ratings': "$book_ratings_users"
    }
  },
    {'$unset':['Ratings._id', 'Ratings.User._id', 'Ratings.User.Geo-Data._id']}
]
merged_cursor=books.aggregate(pipeline)

for i in merged_cursor:
    db['merged'].insert_one(i)
merged=db['merged']
    

In [11]:
print("Example of a record within 'merged' collection:")
pprint(merged.find_one({}))

Example of a record within 'merged' collection:
{'Book-Author': 'Mark P. O. Morford',
 'Book-Title': 'Classical Mythology',
 'ISBN': 195153448,
 'Publisher': 'Oxford University Press',
 'Ratings': [],
 'Year-Of-Publication': 2002,
 '_id': ObjectId('6572995083972156e2c7a505')}


In [12]:
db.list_collection_names()

['merged',
 'books',
 'merged_ratings_users',
 'merged_users_geo',
 'ratings',
 'geodata',
 'users']